In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
drive_path = '/Users/Oruganti/Library/CloudStorage/GoogleDrive-finmetrics0@gmail.com/My Drive'
root = f'{drive_path}/Fin Labs/data/supplement data/money_control/financials'
stat_folder = f'{root}/10-2022'

In [3]:
tickers = pd.read_csv(f'{root}/mc_tickers.csv')
tickers = tickers[tickers['industry_group'].str.contains('Bank')]
tickers = tickers[~tickers['industry_sub_group'].str.contains('Other Bank')]
companies = [f'{c}.xlsx' for c in tickers['security_name']]
len(companies)

31

In [4]:
def extract_info(sheet_name, attributes, n):
    missing = []
    dfs = []
    for c in companies:
        xl = pd.ExcelFile(f'{stat_folder}/{c}')
        if sheet_name[0] in xl.sheet_names:
            sheet = sheet_name[0]
        elif sheet_name[1] in xl.sheet_names:
            sheet = sheet_name[1]
        else:
            sheet = ''
        
        if sheet != '':
            try:
                df = pd.read_excel(f'{stat_folder}/{c}', sheet_name = sheet, index_col = 0)
                df = df.loc[attributes, df.columns[:n]]
                if df.empty:
                    missing.append(c)
                else:
                    df.reset_index(inplace = True)
                    df.rename(columns = {df.columns[0] : 'index'}, inplace = True)
                    df.index = [c] * df.shape[0]
                    dfs.append(df)
            except:
                missing.append(c)
                
    df = pd.concat(dfs)
    return missing, df

#### Intensity of borrowing
Customer deposits are cheapest borrowing vehicle for the banks, however they borrow from other banks at a higher
rate to meet the growth demand. The fraction of borrowing on deposits is a measurement is a relative measure provided 
the deposits are significant enough

In [5]:
sheet_name = ['consolidated-balance-sheet', 'balance-sheet']
attributes = ['Deposits', 'Borrowings', 'Other Liabilities and Provisions']
n = 5
missing, df = extract_info(sheet_name, attributes, n)
print(f'missing: {len(missing)}')
df['aggregate'] = df.sum(axis= 1)
df.reset_index(inplace = True)
df.rename(columns = {'level_0' : 'name'}, inplace = True)
df['name'] = df['name'].str.replace('.xlsx', '')
df.head(2)

missing: 0


,name,index,Mar 22,Mar 21,Mar 20,Mar 19,Mar 18,aggregate
0,SBI,Deposits,4087410.60,3715331.24,3274160.63,2940541.06,2722178.28,16739621.81
1,SBI,Borrowings,449159.78,433796.21,332900.67,413747.66,369079.34,1998683.66


In [6]:
df['aggregate'] = df['aggregate'].apply(float)
borrow_deposit_ratio = []
for name in df['name'].unique():
    deposits = df[(df['name'] == name) & (df['index'] == 'Deposits')]['aggregate'].values[0]
    borrowings = df[(df['name'] == name) & (df['index'] == 'Borrowings')]['aggregate'].values[0]
    ratio = borrowings / deposits
    borrow_deposit_ratio.append((name, deposits, borrowings, ratio))
df = pd.DataFrame(borrow_deposit_ratio, columns = ['name', 'deposits', 'borrowings', 'agg_borrowings_to_deposits'])
df.sort_values(by = 'deposits', ascending = False, inplace = True)
df.head(5)

,name,deposits,borrowings,agg_borrowings_to_deposits
0,SBI,16739621.81,1998683.66,0.119398
2,HDFC Bank,5748808.86,905672.43,0.157541
5,Bank of Baroda,4317982.45,410269.50,0.095014
15,PNB,4308518.89,286339.86,0.066459
7,ICICI Bank,4119203.32,959080.35,0.232832


In [7]:
df['agg_borrowings_to_deposits'].mean()

0.12972735342595756

In [8]:
df[df['name'] == 'IDFC First Bank']

,name,deposits,borrowings,agg_borrowings_to_deposits
26,IDFC First Bank,377547.84,283416.34,0.750677


##### OBSERVATIONS
IDFC First Bank has the highest borrowings to deposits ratio with over 0.75 on aggregate of previous 5 years. It is expected of the company to incur losses in the early stage of business life cycle, and to cover them they seek borrowers. IDFC First Bank has no expection and now that it is getting profitable, the loan will be paid gradually.

Note that the bank has a strong CASA ratio and they are expected to replace these high cost borrowings with 5-5% rates on customer deposits.

### COST TO INCOME RATIO
Book measure of cost to income ratio is different from IDFC First Bank and the industry itself may follow the same or otherwise. I will use both the meaures for reporting.

In [9]:
sheet_name = ['consolidated-profit-loss', 'profit-loss']
attributes = ['Interest / Discount on Advances / Bills',
                'Income from Investments',
                'Interest on Balance with RBI and Other Inter-Bank funds',
                'Others',
                'Total Interest Earned',
                'Other Income',
                'Total Income',
                'Interest Expended',
                'Payments to and Provisions for Employees',
                'Operating Expenses (excludes Employee Cost & Depreciation)',
                'Total Operating Expenses',
                'Total Provisions and Contingencies',
                'Net Profit / Loss for The Year']

n = 5
missing, df = extract_info(sheet_name, attributes, n)
print(f'missing: {len(missing)}')
df['aggregate'] = df['Mar 22']#.sum(axis = 1)
df.reset_index(inplace = True)
df.rename(columns = {'level_0' : 'name'}, inplace = True)
df['name'] = df['name'].str.replace('.xlsx', '')
df.head(2)

missing: 6


,name,index,Mar 22,Mar 21,Mar 20,Mar 19,Mar 18,aggregate
0,SBI,Interest / Discount on Advances / Bills,177474.83,176780.19,185494.19,166124.58,144958.59,177474.83
1,SBI,Income from Investments,93477.90,87130.62,74812.87,80243.51,75036.62,93477.90


In [10]:
def get_value(df, name, index, col_name = 'aggregate'):
    return df[(df['name'] == name) & (df['index'] == index)]['aggregate'].values[0]
    
data = []
for name in df['name'].unique():
    interest_on_advances = get_value(df, name, 'Interest / Discount on Advances / Bills')
    interst_on_investments = get_value(df, name, 'Income from Investments')
    interest_on_rbi_other_banks = get_value(df, name, 'Interest on Balance with RBI and Other Inter-Bank funds')
    total_income = get_value(df, name, 'Total Income')
    interest_expended = get_value(df, name, 'Interest Expended')
    operating_income = total_income - interest_expended
    operating_expense = get_value(df, name, 'Total Operating Expenses')
    net_profit = get_value(df, name, 'Net Profit / Loss for The Year')

    data.append([name, interest_on_advances, interst_on_investments, 
                 interest_on_rbi_other_banks, total_income, interest_expended, 
                 operating_income, operating_expense, net_profit])
    
pl_df = pd.DataFrame(data, 
                  columns = ['name', 'interest_on_advances', 'interest_on_investments', 
                             'interest_on_rbi_other_banks', 'total_income', 'interest_expense', 
                             'operating_income', 'operating_expense', 'net_profit'])
pl_df.head(2)
        

,name,interest_on_advances,interest_on_investments,interest_on_rbi_other_banks,total_income,interest_expense,operating_income,operating_expense,net_profit
0,SBI,177474.83,93477.90,4608.35,406973.09,156194.34,250778.75,174363.43,36356.17
1,HDFC Bank,106295.34,25907.06,2630.78,167695.40,58584.33,109111.07,40312.43,38150.90


In [11]:
# for i in [0.10, 0.25, 0.50, 0.75, 0.90]:
#     print(f"quantile: {i:.2f} {df['ratio'].quantile(i):.4f}")

#### OBSERVATION
Not suprisingly, IDFC First Bank's cost to income ratio is among the highest in the industry. Can the company reduces the ratio to 0.5 from replacing borrowings with customer deposits?

### INTEREST PRODUCING ASSETS
Assets that produces interests (income) to the bank

In [12]:
sheet_name = ['consolidated-balance-sheet', 'balance-sheet']
attributes = ['Advances', 'Investments', 'Total Assets', 
              'Cash and Balances with Reserve Bank of India', 
              'Balances with Banks Money at Call and Short Notice',
              'Total Share Capital', 'Total Reserves and Surplus']

n = 5
missing, df = extract_info(sheet_name, attributes, n)
print(f'missing: {len(missing)}')
df['aggregate'] = df['Mar 22']#.sum(axis = 1)
df.reset_index(inplace = True)
df.rename(columns = {'level_0' : 'name'}, inplace = True)
df['name'] = df['name'].str.replace('.xlsx', '')
df.head(2)

missing: 0


,name,index,Mar 22,Mar 21,Mar 20,Mar 19,Mar 18,aggregate
0,SBI,Advances,2794076.0,2500598.99,2374311.18,2226853.67,1960118.54,2794076.0
1,SBI,Investments,1776489.9,1595100.27,1228284.28,1119247.77,1183794.24,1776489.9


In [13]:
results = []
for name in df['name'].unique():
    advances = get_value(df, name, 'Advances')
    investments = get_value(df, name, 'Investments')
    with_rbi = get_value(df, name, 'Cash and Balances with Reserve Bank of India')
    with_other_banks = get_value(df, name, 'Balances with Banks Money at Call and Short Notice')
    total_assets = get_value(df, name, 'Total Assets')
    share_capital = get_value(df, name, 'Total Share Capital')
    retained_earnings = get_value(df, name, 'Total Reserves and Surplus')
    equity_capital = share_capital + retained_earnings
    
    results.append([name, advances, investments, with_rbi, with_other_banks, 
                    total_assets, share_capital, retained_earnings, equity_capital])
    
bs_df = pd.DataFrame(results, columns = ['name', 'advances', 'investments', 
                                         'with_rbi', 'with_other_banks', 'total_assets',
                                         'share_capital', 'retained_earnings', 'equity_capital'])
bs_df.head(2)

,name,advances,investments,with_rbi,with_other_banks,total_assets,share_capital,retained_earnings,equity_capital
0,SBI,2794076.00,1776489.9,258086.43,140818.69,5360883.53,892.46,304695.58,305588.04
1,IDBI Bank,145775.33,83475.0,13593.91,13206.56,302356.26,10752.40,31819.31,42571.71


In [14]:
df = pl_df.merge(bs_df, on = 'name', how = 'left')
group_df = tickers[['security_name', 'industry_sub_group']]
group_df.rename(columns = {'security_name': 'name', 'industry_sub_group': 'group'}, inplace = True)
df = df.merge(group_df, on = 'name', how = 'left')
df = df.dropna(axis = 0, thresh  = 9)
df['group'] = df['group'].apply(lambda x: 'public' if 'Public' in x else 'private')
print(df.shape)
df.head(2)

(24, 18)


,name,interest_on_advances,interest_on_investments,interest_on_rbi_other_banks,total_income,interest_expense,operating_income,operating_expense,net_profit,advances,investments,with_rbi,with_other_banks,total_assets,share_capital,retained_earnings,equity_capital,group
0,SBI,177474.83,93477.90,4608.35,406973.09,156194.34,250778.75,174363.43,36356.17,2794076.00,1776489.90,258086.43,140818.69,5360883.53,892.46,304695.58,305588.04,public
1,HDFC Bank,106295.34,25907.06,2630.78,167695.40,58584.33,109111.07,40312.43,38150.90,1420942.28,449263.86,130030.71,25355.02,2122934.30,554.55,246771.62,247326.17,private


In [15]:
# Question 1: Return on advances, investments, and rbi/other banks
df['return_on_advances'] = df['interest_on_advances'] / df['advances']
df['return_on_investments'] = df['interest_on_investments'] / df['investments']
df['return_on_rbi_other_banks'] = df['interest_on_rbi_other_banks'] / (df['with_rbi'] + df['with_other_banks'])
df['roe'] = 100 * df['net_profit'] / df['equity_capital']

In [16]:
df[['name', 'group', 'return_on_advances', 'return_on_investments', 'return_on_rbi_other_banks', 'roe']].\
    sort_values(by = ['group', 'return_on_advances'], ascending = False)

,name,group,return_on_advances,return_on_investments,return_on_rbi_other_banks,roe
18,Punjab & Sind,public,0.075101,0.049542,0.018834,7.416176
16,Indian Bank,public,0.069189,0.062157,0.010648,8.884802
17,Central Bank,public,0.068621,0.065822,0.023318,3.863967
12,Union Bank,public,0.068279,0.057297,0.017902,7.350293
11,PNB,public,0.066607,0.062975,0.017049,3.767050
4,Bank of Baroda,public,0.063655,0.056205,0.008741,8.381834
0,SBI,public,0.063518,0.052619,0.011552,11.897118
13,Bank of Mah,public,0.062755,0.063768,0.004554,8.189801
5,Bank of India,public,0.061436,0.061879,0.006954,6.026199
21,Bandhan Bank,private,0.129651,0.050838,0.015074,0.723715


In [ ]:
# for i in [0.10, 0.25, 0.50, 0.75, 0.90]:
#     print(f"quantile: {i:.2f} {df['total_assets'].quantile(i):.0F}")

### OBSERVATION
IDFC First Bank's assets allocation on advances is compariable to the other leading private banks at around 60 percent. The bank total assets is only within 2L crores. Can the bank grows its assets beyond 5L cr, similar to Indusind bank? Not that there is a skew in total assets between public and private sector banks because government transcations and deposits normally happens within public sector bank.

Can the bank grow its assets to 5L cr (indusind bank) or 10L cr (Axis bank) in 10 years?